In [1]:
from lxml import html
import requests
import pandas as pd 
from urllib.request import urlopen
from urllib.error import HTTPError
from urllib.error import URLError
from bs4 import BeautifulSoup
import subprocess as sp
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders
            
def send_mail(receiver,subject,body,file_list):

    msg=MIMEMultipart()
    sender = 'khalil.jlif@rwth-aachen.de'
    msg['From']=sender
    msg['To']=receiver
    msg['Subject']=subject

    body='hi this is an email from khalil '
    msg.attach(MIMEText(body,'plain'))

    #file_list=['TD2A_Eco_Web_Scraping_corrige (1).pdf','zzzz.txt']
    for filename in file_list:

        attachement=open(filename,'rb')
        part=MIMEBase('application','octet-stream')
        part.set_payload(attachement.read())
        encoders.encode_base64(part)
        part.add_header('Content-Disposition',"attachement; filename= "+ filename)
        msg.attach(part)

        text=msg.as_string()
    server=smtplib.SMTP('mail.rwth-aachen.de',587 )
    server.starttls()

    server.login('nm741271@rwth-aachen.de','fzaxf9bxf9')
    server.sendmail(sender,receiver,text)
    server.quit()

def get_rwth_root(number_pages):
    roots=[]
    page=1
    
    while (page<=number_pages):
        print("page",page,"from",number_pages)
        rwth_url=requests.get('https://www.rwth-aachen.de/cms/root/Die-RWTH/Arbeiten-an-der-RWTH/~buym/Jobboerse/?page='+str(page))
        rwth_text=rwth_url.content
        mensa_soup=BeautifulSoup(rwth_text)
        link_in_page = []
        for a in mensa_soup.findAll("a",{'target':'_self'}):
            root =(a.get('href')) 
            if 'Einzelansicht'in root:
                link_in_page.append(root)
                table=(a.string).upper()
                if ("STUDENTISCHE HILFSKRAFT" in table):
                    roots.append(root)
                    
        if len(link_in_page) == 0:
                break
        page=page+1
    return roots

def check_keywords(stringewi , key_words):
        keywords_list=[] 
        big_string=stringewi.upper()
        for i in key_words :
            if i in big_string : 
                keywords_list.append(i)
        return keywords_list

def name_trim (stringie) : 
    to_trim=['M. Sc.','Dipl.-Wirt.-Ing.','M.Sc.',' M.Sc','  RWTH','Dipl.-Phys.']#'Dipl','Wirt','Ing','-','.','M Sc',
    aaa=stringie
    for j in to_trim:
        if j in stringie :
            aaa=stringie.replace(j,'')
    return aaa

#key_words=['PYTHON','MATLAB','C++','C#','JAVA','ELEKTROTECH','INFORMATIK','REGELUNG','WEB','MODELLIER','MESSTECH','DATA','DIGITALIS']        

def rwth_bewerbung_data(a,key_words):
    
    my_roots=get_rwth_root(a)
    repertory_list=[]
    betreuer_names=[]

    for i in range (len(my_roots)):
        
        print("loading",int(i/len(my_roots)*100),"%")
        my_root_string=(my_roots[i])
        rwth_url_jobs=requests.get(my_root_string)
        rwth_url_text=rwth_url_jobs.content
        rwth_url_text_path=html.fromstring(rwth_url_text)
        soup=BeautifulSoup(rwth_url_text)

        for a in soup.findAll("div",{"class":"text"}):
            
            to_search=str(a.findAll('p'))
            found_keywords=check_keywords(to_search,key_words)
            
            if (len(found_keywords)==0):
                break
            
            betreuer_name=a.find('p',{'class':'fn n'}).contents[0]
            betreuer_names.append(betreuer_name)
            betreuer_mailie=rwth_url_text_path.xpath('//div[@id="main"]//dd[@class="value"]//a/@href')
            betreuer_mailiee=''.join(betreuer_mailie)
            betreuer_mail=betreuer_mailiee.replace('mailto:','')
            adress_xpath='//div[@id="main"]//table[@style="width: 100%;"]//tbody//tr[3]//td[2]/text()'
            adress=rwth_url_text_path.xpath(adress_xpath)

            if len(adress)==0 or len(adress)==1 : 
                
                adress_xpath='//div[@id="main"]//table[@style="width: 100%;"]//tbody//tr[2]//td[2]/text()'
                adress=rwth_url_text_path.xpath(adress_xpath)

            institut_name=rwth_url_text_path.xpath('//div[@id="main"]//div[@class="text"]//p[1]/text()')[-1]
            initial_repertory={'Found keyword':' '.join(found_keywords),'Name':betreuer_name,'Mail':betreuer_mail,'Institute adress':''.join(adress),'Institute name':institut_name ,'Link':my_root_string}
            repertory_list.append(initial_repertory)
    
    bewerbung_infos=pd.DataFrame(repertory_list)
    return bewerbung_infos

bewerbung_infos=rwth_bewerbung_data(10,["DATA"])
bewerbung_infos

page 1 from 10
page 2 from 10
page 3 from 10


KeyboardInterrupt: 

In [14]:
bewerbung_infos

,Found keyword,Name,Mail,Institute adress,Institute name,Link
0,PYTHON,Felix Pütz,felix.puetz@iehk.rwth-aachen.de,Felix PützIntzestraße 152072 Aachen,Lehr- und Forschungsgebiet für Werkstoff- und ...,http://www.rwth-aachen.de/cms/root/Die-RWTH/Ar...
1,PYTHON,Tommy Venek,tommy.venek@ipt.fraunhofer.de,Fraunhofer-Institut für Produktionstechnologie...,Fraunhofer-Institut für Produktionstechnologie...,http://www.rwth-aachen.de/cms/root/Die-RWTH/Ar...
2,PYTHON,M. Sc. Niklas Prätzsch,niklas.praetzsch@ilt.fraunhofer.de,Herrn Niklas PrätzschFraunhofer-Institut für L...,Fraunhofer-Institut für Lasertechnik,http://www.rwth-aachen.de/cms/root/Die-RWTH/Ar...
3,PYTHON,M.Sc. Johannes Mayer,J.Mayer@wzl.rwth-aachen.de,M.Sc. Johannes Mayer WZL der RWTH AachenDat...,Werkzeugmaschinenlabor der RWTH Aachen,http://www.rwth-aachen.de/cms/root/Die-RWTH/Ar...
4,PYTHON,M.Sc. Johannes Mayer,J.Mayer@wzl.rwth-aachen.de,M.Sc. Johannes Mayer WZL der RWTH AachenDat...,Werkzeugmaschinenlabor der RWTH,http://www.rwth-aachen.de/cms/root/Die-RWTH/Ar...
5,PYTHON,Christian Möbitz,christian.moebitz@ita.rwth-aachen.de,Institut für TextiltechnikChristian MöbitzOtto...,Institut für Textiltechnik (ITA),http://www.rwth-aachen.de/cms/root/Die-RWTH/Ar...
6,PYTHON,Sören Münker,s.muenker@wzl.rwth-aachen.de,WZLCampus-Boulevard 3052074 Aachen,Werkzeugmaschinenlabor WZL,http://www.rwth-aachen.de/cms/root/Die-RWTH/Ar...
7,PYTHON,"Pit Heimes, M. Sc.",Pit.Heimes@fir.rwth-aachen.de,"FIR an der RWTH Aachen Pit Heimes, M.Sc. Camp...",FIR an der RWTH Aachen,http://www.rwth-aachen.de/cms/root/Die-RWTH/Ar...
8,PYTHON,"Pit Heimes, M. Sc.",Pit.Heimes@fir.rwth-aachen.de,"FIR an der RWTH Aachen Pit Heimes, M.Sc. Camp...",FIR an der RWTH Aachen,http://www.rwth-aachen.de/cms/root/Die-RWTH/Ar...
9,PYTHON,Ruppert,ruppert@vka.rwth-aachen.de,Lehrstuhl für Verbrennungskraftmaschinenz. Hd....,Institut für Verbrennungskraftmaschinen (VKA),http://www.rwth-aachen.de/cms/root/Die-RWTH/Ar...


In [68]:
#function to trim the name 





In [66]:
bewerb_df = bewerbung_infos.copy()

In [41]:
type(betreuer_names[1])

bs4.element.NavigableString

In [67]:
bewerb_df.columns

Index(['Found keyword', 'Name', 'Mail', 'Institute adress', 'Institute name',
       'Link'],
      dtype='object')

In [74]:
bewerb_df['Name_mod'] = bewerb_df.apply(lambda x:name_trim(x['Name']),axis=1)

In [73]:
bewerb_df

,Found keyword,Name,Mail,Institute adress,Institute name,Link,Name_mod
0,MATLAB MESSTECH,David Wollborn,david.wollborn@avt.rwth-aachen.de,David WollbornAVT.BioVTForckenbeckstraße 51520...,Lehrstuhl für Bioverfahrenstechnik AVT.BioVT,http://www.rwth-aachen.de/cms/root/Die-RWTH/Ar...,David Wollborn
1,REGELUNG MESSTECH,M. Sc. Jörg Hofmann,joerg.hofmann@tos.rwth-aachen.de,Herr Jörg HofmannLehrstuhl für Technologie Opt...,Lehrstuhl für Technologie Optischer Systeme,http://www.rwth-aachen.de/cms/root/Die-RWTH/Ar...,Jörg Hofmann
2,ELEKTROTECH INFORMATIK MESSTECH,Dr.-Ing. Christopher Brandl,c.brandl@iaw.rwth-aachen.de,Institut für ArbeitswissenschaftDr.-Ing. Chris...,Institut für Arbeitswissenschaft,http://www.rwth-aachen.de/cms/root/Die-RWTH/Ar...,Dr.-Ing. Christopher Brandl
3,INFORMATIK DATA DIGITALIS,Dipl.-Wirt.-Ing. Thomas Vollmer,studierende.produktionsqualitaet@ipt.fraunhofe...,Fraunhofer-Institut für Produktionstechnologie...,Fraunhofer-Institut für Produktionstechnologie...,http://www.rwth-aachen.de/cms/root/Die-RWTH/Ar...,Thomas Vollmer
4,WEB,Sebastian Weber M.Sc.,bewerbung.technologieorganisation@ipt.fraunhof...,Fraunhofer-Institut für Produktionstechnologie...,Fraunhofer-Institut für Produktionstechnologie...,http://www.rwth-aachen.de/cms/root/Die-RWTH/Ar...,Sebastian Weber.
...,...,...,...,...,...,...,...
82,MATLAB MODELLIER,Deng,y.deng@iwm.rwth-aachen.de,Yuanbin DengLehrstuhl und Institut für Werksto...,IWM,http://www.rwth-aachen.de/cms/root/Die-RWTH/Ar...,Deng
83,JAVA INFORMATIK WEB,Prof. Dr. Edgar Dahl,edahl@ukaachen.de,RWTH cBMBInstitut für PathologieUniklinik RWTH...,RWTH cBMB,http://www.rwth-aachen.de/cms/root/Die-RWTH/Ar...,Prof. Dr. Edgar Dahl
84,ELEKTROTECH INFORMATIK,"Sascha Thamm, M.Sc.",s.thamm@wzl.rwth-aachen.de,Werkzeugmaschinenlabor WZL der RWTH AachenCamp...,Werkzeugmaschinenlabor WZL der RWTH Aachen,http://www.rwth-aachen.de/cms/root/Die-RWTH/Ar...,"Sascha Thamm,."
85,MESSTECH,Dr.-Ing. Sascha Brose,sascha.brose@tos.rwth-aachen.de,Dr.-Ing. Sascha BroseLehrstuhl für Technologie...,Lehrstuhl für Technologie Optischer Systeme,http://www.rwth-aachen.de/cms/root/Die-RWTH/Ar...,Dr.-Ing. Sascha Brose


In [6]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders


def send_mail(receiver,subject,body,file_list):

    msg=MIMEMultipart()
    sender = 'khalil.jlif@rwth-aachen.de'
    msg['From']=sender
    msg['To']=receiver
    msg['Subject']=subject

    body='hi this is an email from khalil '
    msg.attach(MIMEText(body,'plain'))

    #file_list=['TD2A_Eco_Web_Scraping_corrige (1).pdf','zzzz.txt']
    for filename in file_list:

        attachement=open(filename,'rb')
        part=MIMEBase('application','octet-stream')
        part.set_payload(attachement.read())
        encoders.encode_base64(part)
        part.add_header('Content-Disposition',"attachement; filename= "+ filename)
        msg.attach(part)

        text=msg.as_string()
    server=smtplib.SMTP('mail.rwth-aachen.de',587 )
    server.starttls()

    server.login('nm741271@rwth-aachen.de','fzaxf9bxf9')
    server.sendmail(sender,receiver,text)
    server.quit()

In [7]:
send_mail('khaliltounsi@hotmail.fr','you are trash','hi sir you are trash', ['zzzz.txt','TD2A_Eco_Web_Scraping_corrige (1).pdf'])

### Extract Ihre Profil / Aufgaben

In [4]:
template_string = 'http://www.rwth-aachen.de/cms/root/Die-RWTH/Arbeiten-an-der-RWTH/Jobboerse/~kbag/JOB-Einzelansicht/file/29889/'
rwth_url_jobs=requests.get(template_string)
rwth_url_text=rwth_url_jobs.content
rwth_url_text_path=html.fromstring(rwth_url_text)
soup=BeautifulSoup(rwth_url_text)

for a in soup.findAll("div",{"class":"text"}):
    p = a.findAll("h3")
    profil_tag = p[10]
    aufgabe_tag = p[11]
    print(p[p.index(profil_tag)].next_sibling)
    print(p[p.index(aufgabe_tag)].next_sibling)

<p>Gute Kenntnisse im EDV-Bereich, vor allem Microsoft Windows.<br/><br/>
Guter Umgang mit den gänigen Officeprogrammen, vorteilhaft sind Kenntnisse in Vectorworks und ArcGis.<br/><br/>
Motivation, sich in neue Themenfelder einzuarbeiten<br/>
Zuverlässigkeit und Gewissenhaftigkeit</p>
<p>allgemeine Administrative Tätigkeiten,<br/>
Beschaffung von EDV-Geräten und deren Verwaltung mit der gängigen Microsoft Windows Software.</p>


In [41]:
#print("loading",int(i/len(my_roots)*100),"%")
my_roots=get_rwth_root(2)
for my_root_string in my_roots[:5]:
    #my_root_string=(my_roots[0])
    rwth_url_jobs=requests.get(my_root_string)
    rwth_url_text=rwth_url_jobs.content
    rwth_url_text_path=html.fromstring(rwth_url_text)
    soup=BeautifulSoup(rwth_url_text)

    for a in soup.findAll("div",{"class":"text"}):
        p = a.findAll("h3")
        #print(p)
        print('Profil: \n')
        print(p[p.index(profil_tag)].next_sibling)
        print('Aufgaben: \n')
        print(p[p.index(aufgabe_tag)].next_sibling)

page 1 from 2
page 2 from 2
Profil: 

<p>- Sie haben Spaß an Programmierung und Simulation <br/>
- Sie streben eine Beschäftigungsdauer von mindestens einem halben Jahr an<br/>
- Sie sind Studierender der Fachrichtungen Maschinenbau, CES oder vergleichbarer Studiengänge<br/>
- Sie verfügen über Grundkenntnisse im Umgang mit Unix-basierten Betriebssystemen</p>
Aufgaben: 

<p>- Verwendung verschiedener leistungsfähiger Bibliotheken zur Netzgenerierung von komplexen Domänen in 4D<br/><br/>
Bitte beachten Sie auch die Ausschreibung auf der Institushomepage: http://www.cats.rwth-aachen.de/</p>
Profil: 

<p>- eingeschriebener Student (w/m/d) an einer Hoch- oder Fachhochschule<br/>
- sehr gute Deutsch- und gute Englischkenntnisse in Wort und Schrift <br/>
- gute Kenntnisse in MS-Office <br/>
- hohes Engagement und eine strukturierte, selbstständige Denk- und Arbeitsweise</p>
Aufgaben: 

<p>- administrative Tätigkeiten in der Verwaltung<br/>
- Unterstützung des Personalwesens</p>
Profil: 

<p>